In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
wiki.name

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="dolphin-llama3"))
retriever = vectordb.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [ ]:
retriever_tool.name

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import  ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

In [ ]:
tools = [wiki, arxiv, retriever_tool]

In [ ]:
tools

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="dolphin-llama3")

In [ ]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

# from langchain import hub
# prompt = hub.pull("klcoder/mistral-functioncalling")
# prompt

In [ ]:
# Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)


In [ ]:
# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})